## Задание 1
серийная корреляция — тест Bresuch-Godfrey

диагностический тест на спецификацию — RESET-TEST, Harvey-Collier тест, Rainbow test

тестирование на гетероскедастичность — Breusch Pagan, White's test

проверка значимости параметра/гипотез — Wald test

## Задание 2

регрессия log(wage) на age IQ SOUTH URBAN

In [59]:
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# stats models: regression fitting
import statsmodels.formula.api as smf
# data visualization
import seaborn as sns
from scipy.stats import t,f 
import statsmodels.api as sm
import io
import re
from statsmodels.iolib.summary2 import summary_params
from statsmodels.iolib.summary2 import summary_col
import scipy.stats as stats
import statsmodels.stats.api as sms # тесты
from scipy.stats import chi2 # 𝜒2-распределение

In [4]:
data1 = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/wage2.csv')
data1.shape

(935, 17)

In [9]:
# Модель 1
# Создаем спецификацию модели через формулу и подгоняем модель
Wage_eq1 = smf.ols(formula = 'np.log(wage)~age+IQ+south+urban', data = data1).fit()
Wage_eq1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           np.log(wage)   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.175
Method:                 Least Squares   F-statistic:                     50.40
Date:                Mon, 29 May 2023   Prob (F-statistic):           1.99e-38
Time:                        21:03:51   Log-Likelihood:                -425.91
No. Observations:                 935   AIC:                             861.8
Df Residuals:                     930   BIC:                             886.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.0871      0.166     30.648      0.000       4.761       5.413
age            0.0234      0.004      5.797      0.000       0.015       0.031
IQ             0.0082      0.001      9.608      0.000       0.007       0.010
south         -0.0962      0.027     -3.543      0.000      -0.150      -0.043
urban          0.1699      0.028      6.073      0.000       0.115       0.225
==============================================================================
Omnibus:                       41.253   Durbin-Watson:                   1.871
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               73.376
Skew:                          -0.323   Prob(JB):                     1.17e-16
Kurtosis:                       4.211   Cond. No.                     1.43e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:

summary_col(Wage_eq1, stars=None)

,np.log(wage)
Intercept,5.0871
,(0.1660)
age,0.0234
,(0.0040)
IQ,0.0082
,(0.0009)
south,-0.0962
,(0.0272)
urban,0.1699
,(0.0280)


In [11]:
print(summary_col(Wage_eq1, stars=True))


               np.log(wage)
---------------------------
Intercept      5.0871***   
               (0.1660)    
age            0.0234***   
               (0.0040)    
IQ             0.0082***   
               (0.0009)    
south          -0.0962***  
               (0.0272)    
urban          0.1699***   
               (0.0280)    
R-squared      0.1782      
R-squared Adj. 0.1746      
Standard errors in
parentheses.
* p<.1, ** p<.05, ***p<.01


In [12]:
data_string = '''№  age IQ south urban 
1  39  115    0    1    
2  27  106    1    1   
3  23  113    1    0  
4  32  102    0    1'''
wn_df = pd.read_csv(io.StringIO(data_string), sep='\s+')
wn_df.drop(columns='№', inplace=True)
wn_df

,age,IQ,south,urban
0,39,115,0,1
1,27,106,1,1
2,23,113,1,0
3,32,102,0,1


In [13]:
Wage_new = pd.DataFrame({'age': [39,27,23,32],
                        'IQ': [115,106,113,102],
                        'south': [0,1,1,0],
                        'urban': [1,1,0,1]})
Wage_new

,age,IQ,south,urban
0,39,115,0,1
1,27,106,1,1
2,23,113,1,0
3,32,102,0,1


In [15]:
# predict log(wage)
LOG_wage = Wage_eq1.predict(Wage_new)
LOG_wage.round(2)

0    7.11
1    6.66
2    6.45
3    6.84
dtype: float64

In [16]:
type(LOG_wage)

pandas.core.series.Series

In [18]:
#Convert pandas Series to DataFrame
# через конструктор
pd.DataFrame({'np.log(wage)': LOG_wage.values}).round(2)

,np.log(wage)
0,7.11
1,6.66
2,6.45
3,6.84


In [19]:
# predict wage
np.exp(LOG_wage).round(2)

0    1224.84
1     780.49
2     635.09
3     934.86
dtype: float64

In [20]:
# predict wage
np.exp(LOG_wage).round(2).to_frame(name='wage')

,wage
0,1224.84
1,780.49
2,635.09
3,934.86


In [21]:
#ответ — для 3 чела зп 635.09

## Задание 3

In [23]:
misha_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Mishkin.csv')
# добавим первые равности к датасету использую метод .diff()
misha_df[['d_log_cpi']] = np.log(misha_df[['cpi']]).diff()
misha_df.head()
# тест отсутствие или наличие корреляции 3 порядка

,Unnamed: 0,pai1,pai3,tb1,tb3,cpi,d_log_cpi
0,1,-3.552289,1.129370,1.100854,1.129406,23.5,NaN
1,2,5.247540,4.001566,1.125513,1.137254,23.6,0.004246
2,3,1.692860,4.492160,1.115715,1.142319,23.6,0.000000
3,4,5.064298,7.817513,1.146380,1.177902,23.7,0.004228
4,5,6.719322,9.433580,1.158520,1.167777,23.8,0.004211


In [26]:
mod_misha = smf.ols(formula='pai3~tb3+d_log_cpi', data=misha_df).fit()
mod_misha.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   pai3   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.644
No. Observations:                 490   F-statistic:                     443.8
Covariance Type:            nonrobust   Prob (F-statistic):          1.85e-110
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6039      0.173      3.483      0.001       0.263       0.945
tb3            0.2480      0.032      7.870      0.000       0.186       0.310
d_log_cpi    584.9371     27.614     21.182      0.000     530.679     639.195
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [39]:
LM, LMpval, F, Fpval = sms.acorr_breusch_godfrey(mod_misha, nlags=3)
LM.round(3), LMpval.round(3)
#тестовая статистика и p-значение

(180.786, 0.0)

10%-критическое значение $\chi^2_{df=3}$

In [38]:
Misha_crit=chi2.ppf(q=1-0.1, df=3)
Misha_crit.round(3)
#Misha_stat=180.786>Misha_crit=6.251 => отвергаем H0 => присутствует серийная корреляция

6.251

тестовая стат<крит статистика => не отвергаем H0 => отсутствует серийная корреляция

In [37]:
#Перепроверка

    LMpval=0.000<chivalue=0.1=>нулевая гипотеза отвергается => присутствует серийная корреляция

## Задание 4

In [42]:
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 #stats models: regression fitting
import statsmodels.formula.api as smf
 # data visualization
import seaborn as sns
from scipy.stats import t 
import statsmodels.api as sm
import io
import re
from statsmodels.iolib.summary2 import summary_params
from statsmodels.iolib.summary2 import summary_col
import scipy.stats as stats
import statsmodels.stats.api as sms # тесты
from scipy.stats import chi2 # 𝜒2-распределение

In [64]:
di_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/diamonds.csv')
di_eq = 'np.log(price)~carat+I(carat**2)+x+y+z'
mod=smf.ols(formula = di_eq, data=di_df).fit()
mod.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(price)   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.933
No. Observations:               53940   F-statistic:                 1.510e+05
Covariance Type:            nonrobust   Prob (F-statistic):               0.00
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         3.8839      0.029    135.007      0.000       3.828       3.940
carat             2.1877      0.032     67.375      0.000       2.124       2.251
I(carat ** 2)    -0.6106      0.007    -92.533      0.000      -0.624      -0.598
x                 0.4155      0.009     44.603      0.000       0.397       0.434
y                 0.0285      0.004      6.413      0.000       0.020       0.037
z                 0.0391      0.007      5.653      0.000       0.026       0.053
=================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:
sms.het_breuschpagan(resid=mod.resid, exog_het=mod.model.exog)

(4712.162203822872, 0.0, 1032.5286166467336, 0.0)

In [66]:
mod_hc=smf.ols(formula=di_eq, data=di_df).fit(cov_type='HC3')
mod_hc.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(price)   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.933
No. Observations:               53940   F-statistic:                 1.520e+05
Covariance Type:                  HC3   Prob (F-statistic):               0.00
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         3.8839      0.274     14.168      0.000       3.347       4.421
carat             2.1877      0.328      6.667      0.000       1.545       2.831
I(carat ** 2)    -0.6106      0.066     -9.313      0.000      -0.739      -0.482
x                 0.4155      0.114      3.652      0.000       0.192       0.639
y                 0.0285      0.097      0.293      0.770      -0.162       0.219
z                 0.0391      0.041      0.965      0.335      -0.040       0.118
=================================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [67]:
alpha=0.01 # Уровень значимости
df = summary_params(mod_hc, alpha=alpha)
df['sign.regressors'] = df.apply(lambda x: 'Значим' if x['P>|t|']<alpha else 'Незначим', axis=1)
df

,Coef.,Std.Err.,t,P>|t|,[0.005,0.995],sign.regressors
Intercept,3.883898,0.274132,14.168000,1.445596e-45,3.177782,4.590015,Значим
carat,2.187715,0.328144,6.666932,2.612064e-11,1.342472,3.032959,Значим
I(carat ** 2),-0.610612,0.065567,-9.312827,1.244745e-20,-0.779500,-0.441723,Значим
x,0.415510,0.113785,3.651729,2.604807e-04,0.122421,0.708600,Значим
y,0.028519,0.097417,0.292755,7.697093e-01,-0.222409,0.279448,Незначим
z,0.039077,0.040509,0.964642,3.347242e-01,-0.065268,0.143421,Незначим


In [49]:
#коэфф при y незначим

In [69]:
mod_hc.fvalue
#робастная тестовая статистика

152029.61568781827

In [68]:
# Критическое значение
alpha=0.01
F_crit =f.ppf(1-alpha, mod.df_model, mod.df_resid)
F_crit.round(2)

3.02

проверка на задаче артамонова

Для набора данных Labour рассмотрим линейную регрессию log(output) на log(capital), log(labour), log(capital)^2, log(labour)^2.

Потестируем значимость влияния labour, т.е. H0:βlog(labour)=βlog2(labour)=0

In [72]:
Labour_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')
specification = 'np.log(output)~np.log(capital)+np.log(labour)+I(np.log(wage)**2)+I(np.log(labour)**2)'
lab=smf.ols(formula = specification, data=Labour_df).fit()
lab.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(output)   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.895
No. Observations:                 569   F-statistic:                     1205.
Covariance Type:            nonrobust   Prob (F-statistic):          1.13e-274
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -2.5967      0.175    -14.838      0.000      -2.940      -2.253
np.log(capital)            0.1431      0.014      9.996      0.000       0.115       0.171
np.log(labour)             0.4066      0.057      7.191      0.000       0.296       0.518
I(np.log(wage) ** 2)       0.1190      0.008     15.584      0.000       0.104       0.134
I(np.log(labour) ** 2)     0.0357      0.006      5.831      0.000       0.024       0.048
==========================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [74]:
lab_hc=smf.ols(formula=specification, data=Labour_df).fit(cov_type='HC3')
lab_hc.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(output)   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.895
No. Observations:                 569   F-statistic:                     773.7
Covariance Type:                  HC3   Prob (F-statistic):          2.40e-227
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -2.5967      0.443     -5.861      0.000      -3.465      -1.728
np.log(capital)            0.1431      0.028      5.164      0.000       0.089       0.197
np.log(labour)             0.4066      0.167      2.435      0.015       0.079       0.734
I(np.log(wage) ** 2)       0.1190      0.012      9.954      0.000       0.096       0.142
I(np.log(labour) ** 2)     0.0357      0.016      2.162      0.031       0.003       0.068
==========================================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [75]:
lab.fvalue, lab_hc.fvalue

(1205.438045553014, 773.6874376403765)

In [85]:
F_stat=lab_hc.f_test('I(np.log(labour) ** 2)=0')
print(F_stat)

<F test: F=4.674070603204485, p=0.031041118992956064, df_denom=564, df_num=1>


In [86]:
# Критическое значение
alpha=0.01
F_crit =f.ppf(1-alpha, 1, 564)
F_crit

6.680024975020216

Вывод: регрессор незначим (робастный F-тест: F_stat < F_crit или P-value > alpha)